In [45]:
#import frequently used modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [46]:
import glob
filenames = glob.glob(r"C:\Users\Niki\Desktop\data_all/*.txt")

In [47]:
filenames

['C:\\Users\\Niki\\Desktop\\data_all\\sms-call-internet-mi-2013-11-01.txt',
 'C:\\Users\\Niki\\Desktop\\data_all\\sms-call-internet-mi-2013-11-02.txt',
 'C:\\Users\\Niki\\Desktop\\data_all\\sms-call-internet-mi-2013-11-03.txt',
 'C:\\Users\\Niki\\Desktop\\data_all\\sms-call-internet-mi-2013-11-04.txt',
 'C:\\Users\\Niki\\Desktop\\data_all\\sms-call-internet-mi-2013-11-05.txt',
 'C:\\Users\\Niki\\Desktop\\data_all\\sms-call-internet-mi-2013-11-06.txt',
 'C:\\Users\\Niki\\Desktop\\data_all\\sms-call-internet-mi-2013-11-07.txt',
 'C:\\Users\\Niki\\Desktop\\data_all\\sms-call-internet-mi-2013-11-08.txt',
 'C:\\Users\\Niki\\Desktop\\data_all\\sms-call-internet-mi-2013-11-09.txt',
 'C:\\Users\\Niki\\Desktop\\data_all\\sms-call-internet-mi-2013-11-10.txt',
 'C:\\Users\\Niki\\Desktop\\data_all\\sms-call-internet-mi-2013-11-11.txt',
 'C:\\Users\\Niki\\Desktop\\data_all\\sms-call-internet-mi-2013-11-12.txt',
 'C:\\Users\\Niki\\Desktop\\data_all\\sms-call-internet-mi-2013-11-13.txt',
 'C:\\Users\

In [48]:
five_files = filenames[:5]

In [49]:
five_files

['C:\\Users\\Niki\\Desktop\\data_all\\sms-call-internet-mi-2013-11-01.txt',
 'C:\\Users\\Niki\\Desktop\\data_all\\sms-call-internet-mi-2013-11-02.txt',
 'C:\\Users\\Niki\\Desktop\\data_all\\sms-call-internet-mi-2013-11-03.txt',
 'C:\\Users\\Niki\\Desktop\\data_all\\sms-call-internet-mi-2013-11-04.txt',
 'C:\\Users\\Niki\\Desktop\\data_all\\sms-call-internet-mi-2013-11-05.txt']

In [50]:
#Set the column names for the data read
col_list = ['gridID', 'timeInterval', 'countryCode', 'smsIn', 'smsOut', 'callIn', 'callOut', 'internet']

In [51]:
def filter_dataframe(df, feature_name, value_set):
    return df[df[feature_name].isin(value_set)]

In [52]:
top10_grid_set = (5161, 5059, 5259, 5061, 5258, 5159, 6064, 4855, 4856, 5262)

In [53]:
hourlyGridActivity = pd.DataFrame()
#Read each tab separated file into Pandas DataFrame
for file in filenames:
    read_data = pd.read_csv(file, sep='\t',header=None, names=col_list, parse_dates=True)
    #Convert timeInterval column which has Epoch timestamps to UTC and then convert to Milan's local timezone and save it to column startTime
    #tz_localize(None) returns local time format instead of "UTC+1:00" format
    read_data['startTime'] = pd.to_datetime(read_data.timeInterval, unit='ms', utc=True).dt.tz_convert('CET').dt.tz_localize(None)
    read_data = filter_dataframe(read_data, 'gridID', top10_grid_set)
    #Drop timeInterval & countryCode columns
    read_data.drop(columns=['timeInterval','countryCode'], inplace=True)
    
    #Groupby gridID and startTime, startTime which is 10 min apart is resampled to hourly aggregation
    read_data_hourly = read_data.groupby(['gridID', pd.Grouper(key='startTime', freq='H')]).sum()
    hourlyGridActivity = pd.concat([hourlyGridActivity,read_data_hourly]).groupby(['gridID', 'startTime']).sum()
    

In [54]:
hourlyGridActivity.head()

smsIn      smsOut     callIn    callOut  \
gridID startTime                                                           
4855   2013-11-01 00:00:00  162.544860  108.047183  86.442554  73.567003   
       2013-11-01 01:00:00   70.501121   49.862977  32.884867  32.880394   
       2013-11-01 02:00:00   70.783845   58.709391  28.995517  20.332120   
       2013-11-01 03:00:00   88.376602   42.572260  10.881794  17.571800   
       2013-11-01 04:00:00   31.449764   13.513663  18.121553  12.697979   

                               internet  
gridID startTime                         
4855   2013-11-01 00:00:00  4889.373920  
       2013-11-01 01:00:00  3699.700209  
       2013-11-01 02:00:00  2826.055092  
       2013-11-01 03:00:00  2704.410485  
       2013-11-01 04:00:00  2736.943809

In [55]:
hourlyGridActivity.shape

(14880, 5)

In [57]:
hourlyGridActivity.to_csv('top10_hourly.csv')

In [58]:
hourlyGridActivity['weekdayFlag'] = hourlyGridActivity.index.get_level_values(1)
hourlyGridActivity['weekdayFlag'] = hourlyGridActivity['weekdayFlag'].dt.weekday

hourlyGridActivity['hours'] = hourlyGridActivity.index.get_level_values(1)
hourlyGridActivity['hours'] = hourlyGridActivity['hours'].dt.hour

hourlyGridActivity['sms'] = hourlyGridActivity['smsIn'] + hourlyGridActivity['smsOut']
hourlyGridActivity['call'] = hourlyGridActivity['callIn'] + hourlyGridActivity['callOut']

In [59]:
#Create a separate dataframe for weekday and weekend activities
hourlyGridActivity_WE = hourlyGridActivity[hourlyGridActivity['weekdayFlag'].isin([5,6])]
hourlyGridActivity_WD = hourlyGridActivity[~hourlyGridActivity['weekdayFlag'].isin([5,6])]


In [60]:
hourlyGridActivity_WE.head()

smsIn     smsOut     callIn    callOut  \
gridID startTime                                                          
4855   2013-11-02 00:00:00  114.131760  64.928537  33.469032  34.289439   
       2013-11-02 01:00:00   50.974548  43.429717  13.136582  16.018380   
       2013-11-02 02:00:00   29.285499  19.808685  11.805004  11.013056   
       2013-11-02 03:00:00   16.223778  12.678175   8.191302   2.865949   
       2013-11-02 04:00:00    8.102809   2.241259   0.582471   1.895735   

                               internet  weekdayFlag  hours         sms  \
gridID startTime                                                          
4855   2013-11-02 00:00:00  3585.156096            5      0  179.060297   
       2013-11-02 01:00:00  3434.976784            5      1   94.404265   
       2013-11-02 02:00:00  2428.445676            5      2   49.094184   
       2013-11-02 03:00:00  2068.043983            5      3   28.901952   
       2013-11-02 04:00:00  1793.040864            5      4   10.344068   

                                 call  
gridID startTime                       
4855   2013-11-02 00:00:00  67.758471  
       2013-11-02 01:00:00  29.154962  
       2013-11-02 02:00:00  22.818059  
       2013-11-02 03:00:00  11.057251  
       2013-11-02 04:00:00   2.478205

In [61]:
#Add hourly metrics - Max, Min & Avg activity volume for each grid in hourly data
gridActivity = pd.DataFrame()
df = pd.DataFrame()
df = hourlyGridActivity_WE.reset_index()

df1 = df.groupby(['gridID','hours'])[['sms']].mean()
df1.reset_index(level=1,drop=True,inplace=True)
hourlyMaxSms_WE = df1.groupby(['gridID'])[['sms']].max()
hourlyMaxSms_WE.columns = ['hourlysmsMax_WE']
hourlyMinSms_WE = df1.groupby(['gridID'])[['sms']].min()
hourlyMinSms_WE.columns = ['hourlysmsMin_WE']
hourlyAvgSms_WE = df1.groupby(['gridID'])[['sms']].mean()
hourlyAvgSms_WE.columns = ['hourlysmsAvg_WE']


df1 = df.groupby(['gridID','hours'])[['call']].mean()
df1.reset_index(level=1,drop=True,inplace=True)
hourlyMaxCall_WE = df1.groupby(['gridID'])[['call']].max()
hourlyMaxCall_WE.columns = ['hourlycallMax_WE']
hourlyMinCall_WE = df1.groupby(['gridID'])[['call']].min()
hourlyMinCall_WE.columns = ['hourlycallMin_WE']
hourlyAvgCall_WE = df1.groupby(['gridID'])[['call']].mean()
hourlyAvgCall_WE.columns = ['hourlycallAvg_WE']

df1 = df.groupby(['gridID','hours'])[['internet']].mean()
df1.reset_index(level=1,drop=True,inplace=True)
hourlyMaxInternet_WE = df1.groupby(['gridID'])[['internet']].max()
hourlyMaxInternet_WE.columns = ['hourlyinternetMax_WE']
hourlyMinInternet_WE = df1.groupby(['gridID'])[['internet']].min()
hourlyMinInternet_WE.columns = ['hourlyinternetMin_WE']
hourlyAvgInternet_WE = df1.groupby(['gridID'])[['internet']].mean()
hourlyAvgInternet_WE.columns = ['hourlyinternetAvg_WE']


df = hourlyGridActivity_WD.reset_index()

df1 = df.groupby(['gridID','hours'])[['sms']].mean()
df1.reset_index(level=1,drop=True,inplace=True)
hourlyMaxSms_WD = df1.groupby(['gridID'])[['sms']].max()
hourlyMaxSms_WD.columns = ['hourlysmsMax_WD']
hourlyMinSms_WD = df1.groupby(['gridID'])[['sms']].min()
hourlyMinSms_WD.columns = ['hourlysmsMin_WD']
hourlyAvgSms_WD = df1.groupby(['gridID'])[['sms']].mean()
hourlyAvgSms_WD.columns = ['hourlysmsAvg_WD']

df1 = df.groupby(['gridID','hours'])[['call']].mean()
df1.reset_index(level=1,drop=True,inplace=True)
hourlyMaxCall_WD = df1.groupby(['gridID'])[['call']].max()
hourlyMaxCall_WD.columns = ['hourlycallMax_WD']
hourlyMinCall_WD = df1.groupby(['gridID'])[['call']].min()
hourlyMinCall_WD.columns = ['hourlycallMin_WD']
hourlyAvgCall_WD = df1.groupby(['gridID'])[['call']].mean()
hourlyAvgCall_WD.columns = ['hourlycallAvg_WD']

df1 = df.groupby(['gridID','hours'])[['internet']].mean()
df1.reset_index(level=1,drop=True,inplace=True)
hourlyMaxInternet_WD = df1.groupby(['gridID'])[['internet']].max()
hourlyMaxInternet_WD.columns = ['hourlyinternetMax_WD']
hourlyMinInternet_WD = df1.groupby(['gridID'])[['internet']].min()
hourlyMinInternet_WD.columns = ['hourlyinternetMin_WD']
hourlyAvgInternet_WD = df1.groupby(['gridID'])[['internet']].mean()
hourlyAvgInternet_WD.columns = ['hourlyinternetAvg_WD']

#Concatenate the weekly average columns to gridActivity dataframe
gridActivity = pd.concat([gridActivity,hourlyMaxSms_WE, hourlyMaxCall_WE, hourlyMaxInternet_WE,
                          hourlyMinSms_WE, hourlyMinCall_WE, hourlyMinInternet_WE,
                          hourlyAvgSms_WE, hourlyAvgCall_WE, hourlyAvgInternet_WE,
                          hourlyMaxSms_WD, hourlyMaxCall_WD,hourlyMaxInternet_WD,
                          hourlyMinSms_WD, hourlyMinCall_WD,hourlyMinInternet_WD,
                          hourlyAvgSms_WD, hourlyAvgCall_WD,hourlyAvgInternet_WD], axis=1)

In [62]:
gridActivity.head()

,hourlysmsMax_WE,hourlycallMax_WE,hourlyinternetMax_WE,hourlysmsMin_WE,hourlycallMin_WE,hourlyinternetMin_WE,hourlysmsAvg_WE,hourlycallAvg_WE,hourlyinternetAvg_WE,hourlysmsMax_WD,hourlycallMax_WD,hourlyinternetMax_WD,hourlysmsMin_WD,hourlycallMin_WD,hourlyinternetMin_WD,hourlysmsAvg_WD,hourlycallAvg_WD,hourlyinternetAvg_WD
gridID,,,,,,,,,,,,,,,,,,
4855,758.237706,674.257690,5708.191036,153.128468,9.427468,2293.742508,483.840753,311.562360,4452.534902,1441.077014,1093.013894,9363.881806,113.750677,6.678943,2480.470265,773.043221,547.203532,6219.270156
4856,708.353639,616.458075,6281.266859,143.090514,10.399367,2171.374303,476.783218,308.939401,4639.802483,1385.519657,1023.377242,8869.305922,114.966854,6.133420,2400.737613,757.660555,525.262337,5895.660534
5059,2290.423209,1972.665168,15101.966231,65.383125,19.125091,1552.894041,909.324261,723.537169,6735.656945,2726.072163,2642.602452,15891.073306,75.401352,10.553157,1704.034352,1231.015313,1146.872085,7822.964561
5061,1247.882128,1042.602769,10826.318092,56.121237,18.245342,1616.887847,524.621078,397.056603,5428.167424,2346.340215,1918.168800,14333.091537,48.273916,9.627117,1662.393806,963.847854,792.310088,6855.460357
5159,1016.152333,897.583130,8196.316380,25.096363,11.530876,1033.805233,404.452216,320.733020,3654.377580,2200.316919,2231.059263,14237.211785,19.213638,5.701194,1138.808722,927.190284,914.044022,6747.331196


In [63]:
gridActivity.shape

(10, 18)